In [0]:
create or refresh streaming table bronze_customers
tblproperties ("quality" = "true")
comment "this is bronze table"
as
select *,_metadata.file_path as input_file_name,
current_timestamp() as ingestion_time
 from cloud_files("/Volumes/circuitbox/landing/operationaldata/customers/","json",map("cloudFiles.inferColumnTypes" , "true"))

In [0]:
create or refresh streaming table silver_customers_clean(
constraint validate_customer_id expect(customer_id is not null) on violation fail update,
constraint validate_customer_name expect(customer_name is not null) on violation drop row,
constraint validate_telephone expect(length(telephone)>= 10),
constraint validate_email expect(email is not null),
constraint validate_date_of_birth expect(date_of_birth >= '1920-01-01')
)
tblproperties ("quality" = "silver")
comment "this is silver table with cleaned table"
as
select 
customer_id,
customer_name,
cast(date_of_birth as date) as date_of_birth,
telephone,
email,
cast(created_date as date) as created_date
from stream(live.bronze_customers)


In [0]:
create or refresh streaming table silver_customers
tblproperties ("quality" = "silver")
comment "this is silver table"



In [0]:
create flow customers_flow as auto cdc into silver_customers
from stream(live.silver_customers_clean)
keys (customer_id)
sequence by created_date
stored as scd type 1